In [ ]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Sklearn:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
# Other:
from scipy.stats import shapiro
from scipy.stats import iqr
from scipy.stats import ttest_rel

# Data:
pd.set_option('display.max_columns', None)
df_co2 = pd.read_excel('Mobiliteitsdata.xlsx')
print("Head of the data")
display(df_co2.head())
print("Column list")
display(df_co2.columns.tolist())
pd.set_option('display.max_columns', 10)

# Deel A: Data exploratie en voorbereiding

1.	Geef voor elke variabele in de dataset weer tot welk meetniveau deze behoort.

| Variabele | Meetniveau |
| --- | --- |
| persoon_ID | Geen |
| geslacht | Nominaal |
| leeftijd | Ratio |
| gewicht_KG | Ratio |
| burgerlijke_staat | Nominaal |
| jaarsalaris_eu | Ratio |
| huishouden_grootte | Ratio |
| stad | Nominaal |
| provincie | Nominaal |
| opleiding_niveau | Ordinaal |
| autobezit | Nominaal |
| fietsbezit | Nominaal |
| elektrisch_auto | Nominaal |
| km_auto_per_jaar | Ratio |
| km_fiets_per_jaar | Ratio |
| km_ov_per_jaar | Ratio |
| km_vliegtuig_per_jaar | Ratio |
| dagen_thuiswerken_per_week_pro_rato | Ratio |
| dagen_kantoorwerken_per_week_pro_rato | Ratio |
| dagen_in_buitenland_per_jaar | Ratio |
| gebruik_carsharing | Ordinaal |
| co2_uitstoot_per_jaar_KG | Ratio |

In [ ]:
# 1
# Gemaakt in markdown. Wellicht dat we al eerder het ID droppen? Deze gaan we toch niet gebruiken
# Mischien ook verstandig om hier ook toelichting bij te geven als dit nodig is?

2.	Waarom is het nodig om vooraf te bekijken tot welk meetniveau elke variabele behoort?

Dit helpt ons om te bepalen welke statistische methode we gaan gebruiken en om te bepalen hoe we de data gaan voorbewerken.<br>
(Mischien nog wat dieper op in gaan)

In [ ]:
# 2

3.	Ga na of er missende/niet kloppende/extreme waarden zijn in de dataset en ga hier op een adequate manier mee om.

In [ ]:
# 3
# import z-score
from scipy.stats import zscore
# Eerst missende waarden:
mv = df_co2.isnull().mean() * 100
print("Missende waarden percentage")
print(mv[mv > 0].sort_values(ascending=False).round(2))

# Extreme waarden
# Instellingen
threshold = 3
columns_to_check = df_co2.select_dtypes(include=[np.number]).columns.difference(['persoon_ID'])

# Loop voor relevante kollomen
for col in columns_to_check:
    z_scores = zscore(df_co2[col].dropna())
    extreme_mask = np.abs(z_scores) > threshold

    if extreme_mask.any():
        print(f"Extreme waarden gedetecteerd in: {col}")
        sns.boxplot(x=df_co2[col])
        plt.title(f"Boxplot van {col}")
        plt.show()

4.	Kies 2 onafhankelijke variabelen waarvan je denkt dat het invloed kan hebben op de jaarlijkse CO2 uitstoot. Visualiseer de verdeling van deze twee variabelen en de verdeling van de afhankelijke variabele.

Twee kolommen die ik heb gekozen met potential correlatie/invloed op "co2_uitstoot_per_jaar_KG" zijn: "km_auto_per_jaar" en "km_vliegtuig_per_jaar". Ik heb voor deze twee kolommen gekozen, omdat deze twee vervoermiddelen worden vaak gebruikt en gebruiken ook veel brandstof.

Op de heatmap kan je zien dat ik had wel gelijk en deze twee variabelen hadden het grootste invloed. In het geval van de auto's er is een heel grote (bijna perfecte) positive correlatie, maar wat verrassing is, is dat hoewel vliegtuigen het tweede hoogste correlatie hierbij hebben die is nog steeds heel klein. Dit is een zwake positive correlatie dat betekent dat gebruiken van vliegtuigen heeft kleine invloed op co2 uitstoot per jaar in kilograms per kilometer. Het is belangrijk om te merken dat vliegtuigen worden gebruikt om grotere afstanden te reisen in kleine tijden en dus terwijl co2 productie per km kan veel minder zijn voor vliegtuigen,

Ik heb voor hist plot gekozen, omdat die laat zien verdelingen goed.
Er zijn geen afhankelijk variabelen met echte invloed (behalve co2 zelf) waarmee de correlatie zou groot zijn (alles minder dan 0.1 corr).

In [ ]:
# 4
# heatmap
numeric_cols = df_co2.select_dtypes(include=['int64', 'float64']).columns
df_numeric = df_co2[numeric_cols]
plt.figure(figsize=(10, 5))
sns.heatmap(df_numeric.corr(), annot=True)
plt.show()

# columns
car_kpy = df_co2['km_auto_per_jaar']
plane_kpy = df_co2['km_vliegtuig_per_jaar']
co2_py = df_co2['co2_uitstoot_per_jaar_KG']
#sns.regplot(x=car_kpy, y=co2_py)
#plt.show()
#sns.regplot(x=plane_kpy, y=co2_py)
#plt.show()

# hist plot
def hist_plot_maker(data, x):
    sns.histplot(data=data, x=x)
    plt.show()
hist_plot_maker(df_co2, car_kpy)
hist_plot_maker(df_co2, plane_kpy)
hist_plot_maker(df_co2, co2_py)

5.	Geef een beschrijving van de 3 visualisaties met behulp van statistische termen.

Bij alle visualizes er is grote hoeveelheid van data met ≈0 waarden, dit heeft een invloed bij de spread en verdeling van de data en betekent ook dat data is niet altijd qua statistieken "netjes" verdeeld. Het is ook de rede waarom ik heb gekozen om IQR in plaats van bereik te gebruiken.

#1 (km_auto_per_jaar): Zonder extreme waarden mean, mode en mediaan zijn in ongeveer hetzelfde plek. Er is geen Skewness.

#2 (km_vliegtuig_per_jaar): Er is grootste getallen op de linkerkant van de plot met kleinste getallen op de rechterkant. Dit geeft aan rechte/positive Skewness van 0.414 (kleine "tail" op rechte kant), met negatieve Kurtosis van -0.34, dus de "peak" is best wel "flat" in vergelijking met normaal verdeeld data.

#3 (co2_uitstoot_per_jaar_KG): Vergelijkbare rechte Skewness met plot #2 (0.440) maar dan met grootse negative Kurtosis, dus te "peak" is nog meer "flatter".



In [ ]:
# 5
# function with column statistics
def show_statistics(var):
    print(var.describe())
    print(f"Median: {var.median()}")
    print(f"Skewness: {var.skew()}")
    print(f"Kurtosis: {var.kurt()}")
    print(f"IQR: {iqr(var)}")
    print(f"Null: {var.isnull().sum()}")
print("--- km_auto_per_jaar ---")
show_statistics(car_kpy)
print("--- km_vliegtuig_per_jaar ---")
show_statistics(plane_kpy)
print("--- co2_uitstoot_per_jaar_KG ---")
show_statistics(co2_py)

6.	Toets, met behulp van statistiek, of de verdelingen (vraag 4) normaal verdeeld zijn.

Ik heb een functie gemaakt die gebruikt shapiro om te berekenen of de verdelingen normaal verdeeld zijn. Dit is niet 100% accuraat maar met berekeningen uit vorige opdracht met statistieken krijg ik een goede beeld van de verdeling. Ik heb hierbij threshold van 0.05 p gezet, omdat dit de norm is in bepalen of de verschil klein genoeg is. Als die p groter is dan de threshold, dan de data is normaal verdeeld.

In [ ]:
# 6
def normaal_verdeeld(var):
    stat, p = shapiro(var)
    print(f"Normaal Verdeeld: {p > 0.05} ({p})")

print("--- km_auto_per_jaar ---")
normaal_verdeeld(car_kpy)
print("--- km_vliegtuig_per_jaar ---")
normaal_verdeeld(plane_kpy)
print("--- co2_uitstoot_per_jaar_KG ---")
normaal_verdeeld(co2_py) # p van 1 kan niet hier kloppen...

7.	Om ervoor te zorgen dat de onafhankelijke variabelen gebruikt kunnen worden voor Machine Learning moeten ze getransformeerd worden. Geef per meetniveau aan wat er moet gebeuren en waarom.

In ider geval de data moet in de vorm van dataframe of numpy array zijn zonder null waarden en numerieke waarden.

Nominaal: Categorische data zonder volgorde moet worden omgezet naar dummyvariabelen, zodat het model de categorieën kan gebruiken zonder te denken dat er een volgorde is.

Ordinaal: Data moet naar numerieke waarden geconverteerd worden (mapping), zodat ze in een model kunnen worden ingevuld en voorspeld worden.

Interval: Standarizatie (Z-scoren) uitvoeren om data op schaal te krijgen, zodat de data gelijk verdeeld is op de schaal bij verschillende variabelen.
Ratio: Hetzelfde als bij Interval.


8.	Voer de transformatie uit. Let op dat je de getransformeerde variabelen in een nieuw dataframe opslaat, zodat je de oorspronkelijke waarden behoudt. De getransformeerde variabelen gebruik je alleen bij deel D. Zorg dat je inzicht krijgt in de relatie tussen de variabelen. Wat zijn je bevindingen?

Ik heb copies gemaakt van dataframes om de organelle niet te veranderen. Daarna heb ik aanpassingen gemaakt op basis van opdracht 1 voor de kolommen.

In [ ]:
# 8
# Dumming Nominaals:
transformed_df_co2 = df_co2.copy()
transformed_df_co2 = pd.get_dummies(transformed_df_co2, columns=['geslacht', 'burgerlijke_staat', 'stad', 'provincie', 'autobezit', 'fietsbezit', 'elektrisch_auto'], drop_first=True)

# Mapping Ordinaals:
print(transformed_df_co2['opleiding_niveau'].unique())
print(transformed_df_co2['gebruik_carsharing'].unique())
transformed_df_co2['opleiding_niveau'] = transformed_df_co2['opleiding_niveau'].map(
    {'MBO': 1, 'HBO': 2, 'WO': 3})
transformed_df_co2['gebruik_carsharing'] = transformed_df_co2['gebruik_carsharing'].map(
    {'Nooit': 1, 'Heel soms': 2, 'Soms': 3, 'Vaak': 4, 'Altijd': 5})

# Standardizing Ratios:
ratio_cols = ['leeftijd', 'gewicht_KG', 'jaarsalaris_eu', 'huishouden_grootte','km_auto_per_jaar', 'km_fiets_per_jaar', 'km_ov_per_jaar', 'km_vliegtuig_per_jaar', 'dagen_thuiswerken_per_week_pro_rato', 'dagen_kantoorwerken_per_week_pro_rato', 'dagen_in_buitenland_per_jaar','co2_uitstoot_per_jaar_KG']

scaler = StandardScaler()
scaled_df_co2 = transformed_df_co2.copy()
scaled_df_co2[ratio_cols] = scaler.fit_transform(scaled_df_co2[ratio_cols])

9.	Welke methode(n) heb je gebruikt en waarom?

Methoden die heb ik hierbij gebruikt zijn .unique(), omdat ik had alle unique waarden van deze kolommen nodig om te beslissen wat moet ik met behulp van .map() mappen voor ordinaal kolommen. Ernaast heb ik StandardScaler() en fit_transform() hier gebruikt om de data te standarizieren (mean in het middle met elke std van 1) voor ratios. get_dummies() is hierbij gebruikt om categorisch data naar numeriek data te veranderen (elke unique waarde krijgt een kolom). Ik heb ook .copy() hier gebruikt om geen vertandingen aan main df te maken, er naas er is ook copy voor scalen gemaakt, omdat er waren twee df nodig.

10.	Is er sprake van multicollineariteit? Waarom zou dit een probleem kunnen zijn en wat zou jouw oplossing zijn?

In [ ]:
# 10
any_corrs = scaled_df_co2.corr()
strong_corrs = []
columns = any_corrs.columns
for i in range(len(columns)):
    for j in range(i+1, len(columns)): # om te voorkomen duplicate
        col1 = columns[i] # eerste kolom
        col2 = columns[j] # tweede kolom
        corr_value = any_corrs.loc[col1, col2] # correlatie waarde
        if abs(corr_value) > 0.5 and abs(corr_value) != 1: # filter en verwijder -
            strong_corrs.append((col1, col2, corr_value))
for col1, col2, corr in strong_corrs:
    print(f"{col1} VS {col2}: {corr:.2f}")

Mannen vs Vrouwen hebben hier perfecte negatieve correlatie coefficient, dat betekent dat als een positieve is dan de andere negatieve wordt (omdat dit is een binary kolom). Er naast km_auto_per jaar heeft sterke negatieve correlatie met autobezit_nee, hetzelfde hoort ook bij km_fiets_per_jaar vs fietsbezit_nee. Deze twee zijn logisch en ook verwacht. Bij predicitev model bouwen een van deze kolommen zou moeten verwijderd zijn om multicollineariteit te voorkomen. Dit zou een probleem zijn, omdat het geeft eigelijk een "weight" aan een kolom terwijl dat helemaal niet nodig is.

Ik heb gekozen om deze for loop te gebruiken om correlaties te vinden van X variabelen. Heatmap was mij eerste idee maar na data transformatie er waren gewoon te veel kolommen.

11.	Bekijk de relatie tussen uitstoot CO2 en de andere onafhankelijke variabelen per geslacht. Kom je hier tot een andere conclusie dan bij vraag 8?

In [ ]:
# 11
x_kolommen = ['km_auto_per_jaar', 'km_vliegtuig_per_jaar']
for col in x_kolommen:
    sns.lmplot(data=df_co2, x=col, y='co2_uitstoot_per_jaar_KG', hue='geslacht')
    plt.title(f'{col} vs CO2 per geslacht')
    plt.xlabel(col)
    plt.ylabel('CO2 per jaar in kg')
    plt.show()

# Deel B: Kansrekenen

12.	Wat is de kans dat een willekeurig persoon 5000 KG CO2 uitstoot? Gebruik de standaard normaal verdeling in de berekening.

In [ ]:
# 12

13.	Op welke manier wordt, bij deze berekening, een van de technieken van calculus toegepast?

In [ ]:
# 13

14.	Is deze kans groter, gelijk of kleiner bij mannen t.o.v. vrouwen?

In [ ]:
# 14

# Deel C: Toetsende statistiek

15.	Stel de hypothesen op.

In [ ]:
# 15

16.	Welke type fouten kunnen er gemaakt worden?

In [ ]:
# 16

17.	Welke statistische toets moet je uitvoeren om te onderzoeken of er een significant verschil is?

In [ ]:
# 17

18.	Voer de toets uit.

In [ ]:
# 18

19.	Geef in je eigen woorden een toelichting op de betekenis van de p-waarde.

In [ ]:
# 19

20.	Welke conclusie kan je trekken?

In [ ]:
# 20

# Deel D: Modelleren

21.	Train een model om de jaarlijkse CO2 uitstoot te voorspellen. Op hoeveel verschillende mogelijkheden kan deze split gemaakt worden?

Deze split kan op veel verschillende manieren gemaakt worden door test_size en random_state hyperparameters te tweaken. Maar verder een van de workflows is om y kolom te droppen voor X, en voor y allen y kolom te gebruiken. Sklearn module (train_test_split) wordt hier gebruikt om splitting automatisch te doen.

In [ ]:
# 21
scaled_df_co2.dropna(inplace=True)
# omdat er zo weinig nulwaarden waren ik heb gekozen om rows met missende waarden te verwijderen
print(scaled_df_co2.isnull().sum()) # geen null waarden
print(scaled_df_co2.dtypes) # numerieke kolommen
print(type(scaled_df_co2)) # dataframe vorm
# Data voldoet nu al aan machine learning criteria
X = scaled_df_co2[['km_auto_per_jaar', 'km_vliegtuig_per_jaar']].values
y = scaled_df_co2['co2_uitstoot_per_jaar_KG'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)


22.	Train een model om de jaarlijkse CO2 uitstoot te voorspellen. Welke metric zou je gebruiken om te toetsen hoe goed je model is en waarom?


Om te kijken hoe goed ons model werkt, zou ik RMSE hier gebruiken, omdat die terug gaat naar originele units en dus geeft aan de beste beeld van de performance van ons model. Er naas R^2 ik hierbij ook handing om te controleren om data op schaal te zetten en evalueren. Ik heb KFold gebruikt om meer accurate evaluatie te hebben. Data die gaan wij voorspellen is een regressie (geen binary data) dus metric systeem zo als confusion matrix of andere methoden die gebruiken True-Positives, True-Negatives, etc., zou hier niet goed kunnen werken.

In [167]:
# 22
X = scaled_df_co2[['km_auto_per_jaar', 'km_vliegtuig_per_jaar']].values
y = scaled_df_co2['co2_uitstoot_per_jaar_KG'].values
kf = KFold(n_splits=5, shuffle=True, random_state=42) # maak 5 folds
reg = LinearRegression() # maak model
rmse_scores = []
r2_scores = []
for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx] # features
    y_train, y_test = y[train_idx], y[test_idx]
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    r2_scores.append(r2_score(y_test, y_pred))
print(f"Mean RMSE: {np.mean(rmse_scores).round(3)}")
print(f"Mean R2: {np.mean(r2_scores).round(3)}")

Mean RMSE: 0.402
Mean R2: 0.837


23.	Geef de wiskunde functie van het model en pas deze toe op jouw eigen situatie (kies 1 groepsgenoot).

$$y = \beta_0 + \beta_1 \cdot x_1 + \beta_2 \cdot x_2$$

y: co2_uitstoot_per_jaar_KG.

$x_1$: km_auto_per_jaar.

$x_2$: km_vliegtuig_per_jaar.

$\beta_0$: Intercept (bias term).

$\beta_1$: Coefficient voor km_auto_per_jaar.

$\beta_2$: Coefficient voor km_vliegtuig_per_jaar.

In [ ]:
# 23
# GenAi: https://grok.com/share/c2hhcmQtMg%3D%3D_a47f4b25-838d-4d26-ab7d-42c72ace49fd
# Voorbeeld data
group_member_data = np.array([[12000, 5000]])  # [[km_auto_per_jaar, km_vliegtuig_per_jaar]]
scaler = StandardScaler()
scaler.fit(scaled_df_co2[['km_auto_per_jaar', 'km_vliegtuig_per_jaar']])
scaled_group_member_data = scaler.transform(group_member_data)
# voorspelling:
predicted_co2 = reg.predict(scaled_group_member_data)

print(f"Voorspelling CO2: {predicted_co2[0]:.2f} kg per jaar")

24. Kies de vijf onafhankelijke variabelen die de meeste invloed hebben op de voorspelling. Train en test nu twee modellen (1: met alle variabelen en 2: met de vijf meest invloedrijke variabelen). Kies 1 metric om de nauwkeurigheden van de modellen te kwantificeren. Gebruik hiervoor 20-fold crossvalidation. Gebruik toetsende statistiek en de resultaten van de 20-fold cross validation om te onderzoeken welk model significant beter is.

In [168]:
# 24
# Functie die maakt 20 folds en bereikent RMSE voor elke split
def rmse_20_reg(X, y):
    kf = KFold(n_splits=20, shuffle=True, random_state=42)
    reg = LinearRegression()
    rmse_scores = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    return rmse_scores

# Top 5 features (X)
co2_corr = scaled_df_co2.corr()['co2_uitstoot_per_jaar_KG'].sort_values(ascending=False)
top5_features = co2_corr.drop('co2_uitstoot_per_jaar_KG').head(5).index.tolist()
print(top5_features)

# Split data in X met alle kolommen behalve y, en andere X met alleen maar top 5 kolommen, en naar y kolom
X_all = scaled_df_co2.drop('co2_uitstoot_per_jaar_KG', axis=1).values
X_top5 = scaled_df_co2[top5_features].values
y = scaled_df_co2['co2_uitstoot_per_jaar_KG'].values

# Gebruik custom functie om data in folds te trainen en testen en berekenen de RMSE van elke fold
rmse_all = rmse_20_reg(X_all, y)
rmse_top5 = rmse_20_reg(X_top5, y)
print(f"Alle kolommen RMSE: {np.mean(rmse_all).round(3)}")
print(f"Top 5 kolommen RMSE: {np.mean(rmse_top5).round(3)}")

# t-test en p-value
t_stat, p_value = ttest_rel(rmse_all, rmse_top5)
print(f"T-test p-value: {p_value.round(3)}")
if p_value < 0.05:
    print("H0 rejected (er is een significant verschil trussen modellen)")
else:
    print("Er is geen statistische verschil")

['km_auto_per_jaar', 'elektrisch_auto_nee', 'km_vliegtuig_per_jaar', 'dagen_in_buitenland_per_jaar', 'km_ov_per_jaar']
Alle kolommen RMSE: 0.267
Top 5 kolommen RMSE: 0.261
T-test p-value: 0.001
H0 rejected (er is een significant verschil trussen modellen)


Ik heb voor RMSE gekozen, omdat die laat het verschil (errors) zien in echte units. Er naast door mij berekening blijkt dat p_value bij t-test kleiner dat 0.05 is, dat betekent dat H0 hypothesis hier rejected wordt. Op basis van dat ik trek een conlusie dat het verschil tussen modellen is statistisch aanwezig.

25.	Splits de target variabele op in 2 klassen, kies hiervoor een geschikte punt. Codeer de eerste groep als 0 en de tweede groep als 1. Welk model zou je nu kunnen gebruiken en waarom?


Ik heb gekozen om data splitsen in het middle van totaal hoeveelheid, daarom heb ik median hier gebruikt om die als criteria van middel te bepalen. Deze split verdeeld y in binare data en dus een logistieke regressie is hierbij mogelijk en aangeraden. Ernaast deze model kan dat geëvalueerd worden met True positive, True negatieve, False positieve en False negatieve op confusion matrix en metric scoren zoals precision, recall en f1.

In [ ]:
# 25
median_co2 = scaled_df_co2['co2_uitstoot_per_jaar_KG'].median()
co2_0 = scaled_df_co2[scaled_df_co2['co2_uitstoot_per_jaar_KG'] <= median_co2]
co2_1 = scaled_df_co2[scaled_df_co2['co2_uitstoot_per_jaar_KG'] > median_co2]
print(co2_0['co2_uitstoot_per_jaar_KG'].describe()) # Ik gebruik hier .describe() om te kijken of waarden kloppen bij mediaan verdeling
print("---" * 100)
print(co2_1['co2_uitstoot_per_jaar_KG'].describe())

26.	Train een nieuw model op de nieuwe target, gebruik hierbij alleen de vijf meest invloedrijke onafhankelijke variabelen die je eerder hebt gekozen. Geef de wiskunde functie van dit model en pas deze toe op jouw eigen situatie (kies 1 groepsgenoot).


In [ ]:
# 26
# Bron: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html
# Maak binary kolom aan op basis van mediaan criteria en split data in X en y om ze te trainen
X = scaled_df_co2[top5_features]
y = (scaled_df_co2['co2_uitstoot_per_jaar_KG'] > median_co2).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# Logistic regressie om binary data te voorspellen:
model = LogisticRegression(max_iter=1000) # max_iter om balans te houden tussen goede voorspellingen en efficient m
model.fit(X_train, y_train)

# Evaluate model accuracy
accuracy = model.score(X_test, y_test)
print(f"Logistic accuracy: {accuracy}") # om te kijken hoe vaak logistic regressie maakt een error, dit is een goede metric systeem hier, omdat data die wordt voorspelt is binary.
print("---" * 100)

# wiskunde functie van model
# GenAI: https://grok.com/share/c2hhcmQtMg%3D%3D_7455cf79-92fe-4373-9ef8-8ee6b232beb4
# Get coefficients en intercept
coefficients = model.coef_[0]
intercept = model.intercept_[0]
print("Wiskundige functie (z):")
print(f"z = {intercept:.3f} + ", end="")
for coef, feature in zip(coefficients, top5_features):
    print(f"{coef:.3f} * {feature} + ", end="")
print("\b\b")

# Groepsgenoot voorbeeld
group_member_data = np.array([[15000, 6.5, 1400, 100, 5]])  # [[km_auto_per_jaar, elektrisch_auto_nee, km_vliegtuig_per_jaar, dagen_in_buitenland_per_jaar, km_ov_per_jaar]]
scaler = StandardScaler()
scaler.fit(scaled_df_co2[top5_features])
scaled_group_member_data = scaler.transform(group_member_data)
# voorspelling:
predicted_co2 = model.predict_proba(scaled_group_member_data)
print(predicted_co2)

Uit het resultaat blijk het dat accuracy voor onze logistic model is heel hoog. Dit is een kenmerk van goed gekozene features voor het trainen van model om y te voorspellen.

27.	Schrijf een conclusie op basis van de twee hoofdvragen van het portfolio (zie kopje ‘Vraagstellingen’)


- Welke persoonlijke en mobiliteitsgerelateerde factoren verklaren verschillen in de jaarlijkse CO₂-uitstoot tussen burgers?

- In welke mate kunnen we individuele uitstoot nauwkeurig voorspellen op basis van een combinatie van sociaaldemografische kenmerken en mobiliteitsgedrag?

In [ ]:
# 27